In [59]:
! pip install sklearn
! pip install xgboost
! pip install networkx
! pip install pandas

Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'


In [60]:
import csv
import networkx as nx
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn import svm, preprocessing, metrics

In [61]:
# 讀取CSV檔
df = pd.read_csv('new_train_data.csv')

# 建立有向圖
G = nx.Graph()
G1 = nx.DiGraph()

# 加入節點
nodes = set(df['node1']).union(set(df['node2']))
G.add_nodes_from(nodes)
G1.add_nodes_from(nodes)


# 加入邊
for index, row in df.iterrows():
    # if row['label'] == 1:
    G.add_edge(row['node1'], row['node2'])
    G1.add_edge(row['node1'], row['node2'])

In [62]:
# 分割訓練集和測試集
train_edges, val_edges, train_labels, val_labels = train_test_split(
    df[['node1', 'node2']].values, df['label'].values, test_size=0.01)

In [63]:
# 提取特徵向量
train_features = []
for edge in train_edges:

    node1 = edge[0]
    node2 = edge[1]
    #print(node1,' ----- ', node2)
    common_neighbors = len(list(nx.common_neighbors(G, node1, node2)))

    # try:
    #     # 可能引發異常的程式碼
    #     shortest_path = len(nx.shortest_path(G1, node1, node2))
    # except:
    #     # 異常處理的程式碼
    #     shortest_path = 1000000
    # indegree = (G1.in_degree(int(node1)) + G1.out_degree(int(node1))) * \
    #     (G1.in_degree(int(node2)) + G1.out_degree(int(node2)))
    preds = nx.jaccard_coefficient(G, [(node1, node2)])
    for u, v, p in preds:
        jaccard_coefficient = p

    preds = nx.preferential_attachment(G, [(node1, node2)])
    for u, v, p in preds:
        preferential_attachment = p

    preds = nx.resource_allocation_index(G, [(node1, node2)])
    for u, v, p in preds:
        resource_allocation_index = p

    train_features.append([common_neighbors, jaccard_coefficient,
                          preferential_attachment, resource_allocation_index])
    #train_features.append([common_neighbors, jaccard_coefficient, indegree, shortest_path])

In [64]:
test_features = []
for edge in val_edges:
    node1 = edge[0]
    node2 = edge[1]
    common_neighbors = len(list(nx.common_neighbors(G, node1, node2)))
    # try:
    #     # 可能引發異常的程式碼
    #     shortest_path = len(nx.shortest_path(G1, node1, node2))
    # except:
    #     # 異常處理的程式碼
    #     shortest_path = 1000000

    indegree = (G1.in_degree(int(node1)) + G1.out_degree(int(node1))) * \
        (G1.in_degree(int(node2)) + G1.out_degree(int(node2)))
    jaccard_coefficient = len(
        list(nx.jaccard_coefficient(G, [(node1, node2)])))
    preds = nx.jaccard_coefficient(G, [(node1, node2)])
    for u, v, p in preds:
        jaccard_coefficient = p

    preds = nx.preferential_attachment(G, [(node1, node2)])
    for u, v, p in preds:
        preferential_attachment = p

    preds = nx.resource_allocation_index(G, [(node1, node2)])
    for u, v, p in preds:
        resource_allocation_index = p
    test_features.append([common_neighbors, jaccard_coefficient,
                         preferential_attachment, resource_allocation_index])
    #test_features.append([common_neighbors, jaccard_coefficient, indegree, shortest_path])

In [65]:
# 訓練模型
clf = XGBClassifier(n_estimators=100, learning_rate=0.2)
clf.fit(train_features, train_labels)

# 預測測試集
test_preds = clf.predict(test_features)

# 計算準確率
accuracy = accuracy_score(val_labels, test_preds)


In [66]:
for row in enumerate(list(val_edges)):
    if train_labels[row[0]] == 1:
        G.add_edge(row[1][0], train_edges[1][1])
        G1.add_edge(row[1][0], train_edges[1][1])


df1 = pd.read_csv('new_test_data.csv')
val_edges1 = df1[['node1', 'node2']].values

nodes1 = set(df1['node1']).union(set(df1['node2']))
G.add_nodes_from(nodes1)
G1.add_nodes_from(nodes1)

for index, row in df1.iterrows():
    G.add_edge(row['node1'], row['node2'])
    G1.add_edge(row['node1'], row['node2'])

test_features1 = []
for node in val_edges1:
    node1 = node[0]
    node2 = node[1]
    common_neighbors = len(list(nx.common_neighbors(G, node1, node2)))
    # try:
    #     # 可能引發異常的程式碼
    #     shortest_path = len(nx.shortest_path(G1, node1, node2))
    # except:
    #     # 異常處理的程式碼
    #     shortest_path = 1000000
    indegree = (G1.in_degree(int(node1)) + G1.out_degree(int(node1))) * \
        (G1.in_degree(int(node2)) + G1.out_degree(int(node2)))
    # print(common_neighbors)
    preds = nx.jaccard_coefficient(G, [(node1, node2)])
    for u, v, p in preds:
        jaccard_coefficient = p

    preds = nx.preferential_attachment(G, [(node1, node2)])
    for u, v, p in preds:
        preferential_attachment = p

    preds = nx.resource_allocation_index(G, [(node1, node2)])
    for u, v, p in preds:
        resource_allocation_index = p
    test_features1.append([common_neighbors, jaccard_coefficient,
                          preferential_attachment, resource_allocation_index])
    #test_features1.append([common_neighbors, jaccard_coefficient, indegree, shortest_path])


test_preds1 = clf.predict(test_features1)
print('Accuracy:', accuracy)
print(test_preds1)
print(sum(test_preds1))

Accuracy: 0.6791666666666667
[1 0 1 ... 1 0 0]
3274


In [67]:
count = 0
n_feature = []
for ff in test_features1:
    ff.insert(0, count)
    count += 1

n_test_features1 = sorted(
    test_features1, key=lambda test_features1: test_features1[3])
n_test_features1.reverse()

result = [0] * 6000
for i in range(3000):
    result[n_test_features1[i][0]] = 1


output = "ans.csv"

with open(output, 'a') as f:
    csv_write = csv.writer(f)
    csv_write.writerow(['node_pair_id', 'ans'])
    index = 0
    for i in test_preds1:
        csv_write.writerow([index, i])
        index += 1
    print("finished")

finished
